# Compiled Deliverable Notebook

In [ ]:
# Import standard packages
import itertools
import numpy as np
import pandas as pd 
pd.options.display.max_rows = 4000
from numbers import Number
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
%matplotlib inline

# Workflow Start: EDA and Transformation of Target Outcome

The first step in the construction of our model is to set a target outcome for our model to predict. Due to the skew that was identified in our initial Exploratory Data Analysis for housing prices as a target outcome, a logarithmic transformation was performed to normalize the distribution and satisfy the linearity assumption needed for linear regression.

In [ ]:
#Read in data file
df = pd.read_csv("./data/kc_house_data.csv")
df

In [ ]:
#selecting outcome and saving as a variable for further use
target = df["price"]

In [ ]:
#visualize if price is normally distributed 
plt.subplots(figsize=(10,10))
sns.histplot(target)

In [ ]:
#check if price is normally distributed 
target.describe()

In [ ]:
#changed setting to display descriptive statistics without scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)
target.describe()


In [ ]:
#possible outliers
target.loc[target>= 6500000]


In [ ]:
#Handle right-skew with log transform and reassess outlier impact on distribution
ln_target = np.log(target)
#visualize  
plt.subplots(figsize=(10,10))
sns.histplot(ln_target)

In [ ]:
#check if price is normally distributed 
ln_target.describe()

In [ ]:
#skew is near zero - note the difference between median and mean in descriptive stats
#check kurtosis
stats.kurtosis(ln_target)

In [ ]:
#since fisher kurtosis score >=0, this distribution is very normal and we will accept this for our target outcome.
#Note that due to this transformation, the final model will be interpreted in terms of logarithm of price rather 
#than simple dollar amounts

#Save new target over old dataframe and rename new dataset
df["price"] = ln_target

#rename
final_df = df.rename(columns={"price":"ln_price"})

#test
final_df

In [ ]:
#save new dataset as csv
final_df.to_csv("./data/ln_price_dataframe", index=False)
#proceed to further eda in further notebooks

# Categorical Feature Cleaning and Preparation

In [ ]:
# Import the data
df = pd.read_csv('./data/kc_house_data.csv')
final_df = pd.read_csv('./data/ln_price_dataframe')

### Column Names and Descriptions for King County Data Set
* `id` - Unique identifier for a house
* `date` - Date house was sold
* `price` - Sale price (prediction target)
* `bedrooms` - Number of bedrooms
* `bathrooms` - Number of bathrooms
* `sqft_living` - Square footage of living space in the home
* `sqft_lot` - Square footage of the lot
* `floors` - Number of floors (levels) in house
* `waterfront` - Whether the house is on a waterfront
  * Includes Duwamish, Elliott Bay, Puget Sound, Lake Union, Ship Canal, Lake Washington, Lake Sammamish, other lake, and river/slough waterfronts
* `view` - Quality of view from house
  * Includes views of Mt. Rainier, Olympics, Cascades, Territorial, Seattle Skyline, Puget Sound, Lake Washington, Lake Sammamish, small lake / river / creek, and other
* `condition` - How good the overall condition of the house is. Related to maintenance of house.
  * See the [King County Assessor Website](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r) for further explanation of each condition code
* `grade` - Overall grade of the house. Related to the construction and design of the house.
  * See the [King County Assessor Website](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r) for further explanation of each building grade code
* `sqft_above` - Square footage of house apart from basement
* `sqft_basement` - Square footage of the basement
* `yr_built` - Year when house was built
* `yr_renovated` - Year when house was renovated
* `zipcode` - ZIP Code used by the United States Postal Service
* `lat` - Latitude coordinate
* `long` - Longitude coordinate
* `sqft_living15` - The square footage of interior housing living space for the nearest 15 neighbors
* `sqft_lot15` - The square footage of the land lots of the nearest 15 neighbors

###### Data Exploration

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
final_df.isna().sum()

In [ ]:
df['sqft_living'].value_counts()

In [ ]:
df.corr()['price'].map(abs).sort_values(ascending=False)

In [ ]:
df['zipcode'].hist(bins=35)

Price is most strongly correlated with sqft_living and sqft_above

In [ ]:
duplicates = final_df[final_df.duplicated(['id','date'])].sort_values(by='id')

In [ ]:
duplicates

In [ ]:
# Here you run your code to clean the data
final_df.select_dtypes(include='object')

#### date

In [ ]:
final_df['date'] = pd.to_datetime(final_df['date'])
final_df['date'].sort_values().max()

This columns can help our stakeholder to understand when the sale prices were recorded, but won't help us much our model; therefore, we will omit it from our analysis.

#### waterfront

In [ ]:
final_df.isna().sum()

In [ ]:
# Change YES/NO values to True/False
final_df['waterfront'].replace({'YES': True, 'NO': False}, inplace=True)

In [ ]:
final_df.loc[final_df['waterfront'] == True].describe()

Given the small amount of trues and large amount of nulls, we are going to assume this column was not recorded well and omit it from our model.

##### view

In [ ]:
final_df['view'].isna().sum()

In [ ]:
final_df['view'].value_counts()

Given the 63 nulls and the majority of values being None, we are going to omit this column from our model

#### condition

Relative to age and grade. Coded 1-5.

1 = Poor- Worn out. Repair and overhaul needed on painted surfaces, roofing, plumbing, heating and numerous functional inadequacies. Excessive deferred maintenance and abuse, limited value-in-use, approaching abandonment or major reconstruction; reuse or change in occupancy is imminent. Effective age is near the end of the scale regardless of the actual chronological age.

2 = Fair- Badly worn. Much repair needed. Many items need refinishing or overhauling, deferred maintenance obvious, inadequate building utility and systems all shortening the life expectancy and increasing the effective age.

3 = Average- Some evidence of deferred maintenance and normal obsolescence with age in that a few minor repairs are needed, along with some refinishing. All major components still functional and contributing toward an extended life expectancy. Effective age and utility is standard for like properties of its class and usage.

4 = Good- No obvious maintenance required but neither is everything new. Appearance and utility are above the standard and the overall effective age will be lower than the typical property.

5= Very Good- All items well maintained, many having been overhauled and repaired as they have shown signs of wear, increasing the life expectancy and lowering the effective age with little deterioration or obsolescence evident with a high degree of utility.

In [ ]:
final_df['condition'].value_counts()

###### Ordinal Encoder

Since the order of the categorical values matters (we expect a Very Good home to be higher in value than a Poor home), we will try to Ordinal Encode this category.

In [ ]:
# Set up Ordinal Encoder
condition = final_df['condition']
condition_df = pd.DataFrame(condition)
#cond_cat = [list(final_df['condition'].value_counts().keys())]
cond_cat = [['Poor', 'Fair', 'Average', 'Good', 'Very Good']]
ords_cond = OrdinalEncoder(categories=cond_cat)
ords_cond.fit(condition_df)
X_cond_transform = ords_cond.transform(condition_df)

In [ ]:
# Run the regression
y = final_df['ln_price']
X_condition_ord = sm.add_constant(X_cond_transform)
X_condition_ord_results = sm.OLS(y, X_condition_ord).fit().summary()
X_condition_ord_results

In [ ]:
# Create a violin plot to understand correlation
X_iterable = [i[0] for i in X_cond_transform]
sns.violinplot(y=y, x=X_iterable);

In [ ]:
# See how correlated Condition is with Price
cond_transform_df = pd.DataFrame(X_cond_transform)
pd.concat([y, cond_transform_df], axis=1).corr()

As shown by the very low rsquared value and low correlation value, these two variables aren't very correlated; however, the pvalue is lower than our alpha which means the predictor is statistically significant. To check, we're going to One Hot Encode these values to see if a specific categorial variable is very correlated to price.

In [ ]:
# Convert array into dataframe
X_condition_ord_df = pd.DataFrame(X_cond_transform, columns=['cond_ord'])
X_condition_ord_df

##### One Hot Encoding

In [ ]:
ohe_cond = OneHotEncoder(drop='first') # drops average, first and most frequent
ohe_cond.fit(condition_df)
condition_hot = pd.DataFrame(ohe_cond.transform(condition_df).todense(), columns=ohe_cond.get_feature_names())
#condition_encoded

In [ ]:
# Run the regression
X_cond_hot = sm.add_constant(condition_hot)
X_condition_hot_results = sm.OLS(y, X_cond_hot).fit().summary()
X_condition_hot_results

As shown above, this model has a higher rsquared meaning it accounts for more variance in the target variable (only slighlty). Given the results, we will most likely use the One Hot Encoded condition variable in our model

#### grade

Represents the construction quality of improvements. Grades run from grade 1 to 13. Generally defined as:

1-3 Falls short of minimum building standards. Normally cabin or inferior structure.

4 Generally older, low quality construction. Does not meet code.

5 Low construction costs and workmanship. Small, simple design.

6 Lowest grade currently meeting building code. Low quality materials and simple designs.

7 Average grade of construction and design. Commonly seen in plats and older sub-divisions.

8 Just above average in construction and design. Usually better materials in both the exterior and interior finish work.

9 Better architectural design with extra interior and exterior design and quality.

10 Homes of this quality generally have high quality features. Finish work is better and more design quality is seen in the floor plans. Generally have a larger square footage.

11 Custom design and higher quality finish work with added amenities of solid woods, bathroom fixtures and more luxurious options.

12 Custom design and excellent builders. All materials are of the highest quality and all conveniences are present.

13 Generally custom designed and built. Mansion level. Large amount of highest quality cabinet work, wood trim, marble, entry ways etc.

In [ ]:
final_df['grade'].value_counts()

###### Ordinal Encoder

Since the order of the categorical values matters (we expect a Very Good home to be higher in value than a Poor home), we will try to Ordinal Encode this category.

In [ ]:
# Set up Ordinal Encoder
grade = final_df['grade']
grade_df = pd.DataFrame(grade)
grade_cat = [['3 Poor', '4 Low', '5 Fair', '6 Low Average', '7 Average', '8 Good', '9 Better', '10 Very Good', '11 Excellent', '12 Luxury', '13 Mansion']]
ords_grade = OrdinalEncoder(categories=grade_cat)
ords_grade.fit(grade_df)
X_grade_transform = ords_grade.transform(grade_df)

In [ ]:
# Run the regression
X_grade_ord = sm.add_constant(X_grade_transform)
X_grade_ord_results = sm.OLS(y, X_grade_ord).fit().summary()
X_grade_ord_results

Notes about regression above

In [ ]:
# Convert array into dataframe
X_grade_ord_df = pd.DataFrame(X_grade_transform, columns=['grade_ord'])
X_grade_ord_df

##### One Hot Encoding

In [ ]:
# Set up One Hot Encoder
ohe_grade = OneHotEncoder(drop='first') # drops average, first and most frequent
ohe_grade.fit(grade_df)
grade_hot = pd.DataFrame(ohe_grade.transform(grade_df).todense(), columns=ohe_grade.get_feature_names())
#grade_encoded

In [ ]:
# Run the regression
X_grade_hot = sm.add_constant(grade_hot)
X_grade_hot_results = sm.OLS(y, X_grade_hot).fit().summary()
X_grade_hot_results

Notes about regression above

#### sqft_basement

In [ ]:
# change sqft_basement data type to numeric
#final_df['sqft_basement'].replace('?', 0, inplace=True)
#final_df['sqft_basement'] = pd.to_numeric(df['sqft_basement'])

##### Create CSV's for Modeling

To use both condition and grade in our models, we are going to create csv's - combining the ordinal encoded data and ohe data into separate dataframes

In [ ]:
# Concat Ordinal Encdoded Dataframes from condition and grade into one dataframe
X_cat_ordinal_df = pd.concat([X_condition_ord_df, X_grade_ord_df], axis=1)
X_cat_ordinal_df

In [ ]:
# Convert ordinal dataframe into a csv
X_cat_ordinal_df.to_csv("./data/cat_ordinal_dataframe", index=False)

In [ ]:
# Concat Ordinal Encdoded Dataframes from condition and grade into one dataframe
X_cat_hot_df = pd.concat([condition_hot, grade_hot], axis=1) 
X_cat_hot_df

Important to note: From the condition table, "Average" value was dropped. From the grade table, "10 Very Good" was dropped. We defaulted to dropping the first value in each category.

In [ ]:
# Convert ohe dataframe into a csv
X_cat_hot_df.to_csv("./data/cat_hot_dataframe", index=False)

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

##### Since sqft_living is the variable most correlated to price (0.70), we will start by creating a simple linear regression between the two

In [ ]:
# set up the simple linear regression between price and sqft_living
simple_endog = final_df['ln_price']
simple_exog_sqftliving = sm.add_constant(df['sqft_living'])

simple_model = sm.OLS(simple_endog, simple_exog_sqftliving).fit().summary()

In [ ]:
# run the simple linear regression model
simple_model

This model only explains 49.3% of the target variable variance, which is quite low. The predicor variable however is significant given that it's pvalue is smaller than our .05 alpha. A one unit increase in sqft_living will increase the price of a home by $280.86. 

##### Let's test out another highly correlated variable (0.61), sqft_above

In [ ]:
final_df.corr()

In [ ]:
# set up the simple linear regression between price and sqft_above
simple_exog_bedroom = sm.add_constant(final_df['bedrooms'])

simple_model_2 = sm.OLS(simple_endog, simple_exog_bedroom).fit().summary()

In [ ]:
# run the simple linear regression model
simple_model_2

As expected, this model explains less variance than the last at 36.6%. Sqft_above is also statistically significant, with a pvalue less than an alpha of .05. A one unit increase in sqft_above will increase the price of a home by $268.67.

# Numeric Data Cleaning and Preparation

In [ ]:
# Import the data
df = pd.read_csv("./data/ln_price_dataframe")


# Investigating the data

In [ ]:
#checking import of data
df                  

In [ ]:
df.info()

In [ ]:
#categorical data being handled in separate notebook - keeping only numeric data
df1 = df.drop(df[['date', 'view', 'waterfront','condition','grade','zipcode','lat', 'long' ]], axis = 1)
df1.head()



In [ ]:
df1.shape

In [ ]:
#checking data types
df1.info()

Converting non-numeric data types into numeric data types for later regression

In [ ]:
df1['sqft_basement'] = df1['sqft_basement'].replace('?', '0.0')
df1['sqft_basement'] = pd.to_numeric(df1['sqft_basement'], errors='coerce')
df1['sqft_basement']

In [ ]:
df1['sqft_basement'].value_counts()

In [ ]:
#check data types 
df1.info()

# Checking Distributions

In [ ]:
#visualize distributions
for a, column in enumerate(df1.columns):
    plt.figure(a)
    sns.distplot(df1[column])

Note the following: 
- severe outliers apparent in sqft_lot15, sqft_basement, sqft_lot
- right tail skew in sqft_living15, sqft_above, sqft_above
- bimodal distribution in yr_renovated with lots of null values
- left tail skew in yr_built, possible multimodality due to spikes in housing construction

*skewed distributions may require transformation to satisfy linear relationship assumption of linear regression. 

*floors, bedrooms, and bathrooms may be usable in their current state with minimal cleaning needed

### convert yr_renovated into boolean data type indicating a house has been renovated or not

In [ ]:
for i,r in enumerate(df1["yr_renovated"]):
    if r == 0:
        df1["yr_renovated"][i] = False
    else:
        df1["yr_renovated"][i] = True

In [ ]:
df1

In [ ]:
#rename yr_renovated
df1.rename(columns={"yr_renovated": "renovated"}, inplace=True)
df1.head()

In [ ]:
#one hot encode renovation data
ohe_df = pd.DataFrame(df1["renovated"])
ohe = OneHotEncoder(drop='first') # drops average, first and most frequent
ohe.fit(ohe_df)
renovation_df = pd.DataFrame(ohe.transform(ohe_df).todense(), columns=ohe.get_feature_names())


In [ ]:
renovation_df

In [ ]:
#linreg on ohe, assumption check 
sm.OLS(df["ln_price"], sm.add_constant(renovation_df)).fit().summary()

In [ ]:
df1["renovated"] = renovation_df["x0_True"]
df1

### Convert Year of Construction to Age

In [ ]:
df1["yr_built"].isna().sum()

In [ ]:
df1["yr_built"] = 2015 - df1["yr_built"]
df1

In [ ]:
df1.rename(columns={"yr_built":"age"}, inplace=True)
df1.columns

### Taking a closer look at sqft_lot15, sqft_basement, sqft_lot

In [ ]:
#general look at central tendency of the 3 features with severe outliers
df1["sqft_lot15"].describe(), df1["sqft_basement"].describe(), df1["sqft_lot"].describe()

severe outliers present in sqft_lot15 - note the order of magnitude difference between 75th percentile and max

large presence of nulls in sqft_basement

sqft_lot similar to sqft_lot15 but with greater severity - max is two orders larger than 75th percentile

# Test for Linearity Assumption

In [ ]:
df1

In [ ]:
#drop basement sqft since majority of houses in king county do not have basements

df1 = df1.drop(labels="sqft_basement", axis=1)
df1

In [ ]:
#visualize scatter plot of each variable against the target outcome to check for linearity
target = df1["ln_price"]
inputs = df1.drop(labels=["id","ln_price"], axis=1)
for a, column in enumerate(inputs.columns):
    plt.figure(a)
    sns.scatterplot(y=target, x=inputs[column])

sqft_above, sqft_living, possibly sqft_living15, are non-linear - need transform? or non-logarithmic target

sqft_lot15 needs nulls addressed?

In [ ]:
#no nulls found
df1["sqft_lot15"].describe()

In [ ]:
#natural log transform sqft_above - attempt to address non-linearity
inputs["sqft_above"] = np.log(inputs["sqft_above"])
sns.scatterplot(y=target, x=inputs["sqft_above"])

In [ ]:
#natural log transform sqft_living - attempt to address non-linearity
inputs["sqft_living"] = np.log(inputs["sqft_living"])
sns.scatterplot(y=target, x=inputs["sqft_living"])

In [ ]:
#natural log transform sqft_living15 - attempt to address non-linearity
inputs["sqft_living15"] = np.log(inputs["sqft_living15"])
sns.scatterplot(y=target, x=inputs["sqft_living15"])

In [ ]:
#rename all affected columns to reflect the changes above
inputs.rename(columns={"sqft_above":"ln_sqft_above", 
                       "sqft_living":"ln_sqft_living",
                       "sqft_living15":"ln_sqft_living15"},
             inplace=True)
inputs

In [ ]:
#save initial model inputs and natural log target prices for use in final constructor notebook via csv
inputs.to_csv("./data/initial_numeric_inputs", index=False)
target.to_csv("./data/house_price_target_natlog", index=False)

# Final Model Construction

Below is the construction for a multiple linear regression model predicting housing prices in King County. Data is imported from the preprocessed data that was cleaned in preceding notebooks and saved as separate csv files. The data is imported to this notebook and combined for the first model. Further iterations are contained in additional subsections of this notebook.

### Import Packages

In [ ]:
import scipy.stats as stats
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import median_absolute_error, mean_squared_error 
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, RobustScaler
from sklearn.linear_model import LinearRegression

### Import and Combine Cleaned Data

In [ ]:
categorical_ohe = pd.read_csv("./data/cat_hot_dataframe")
categorical_ordinal = pd.read_csv("./data/cat_ordinal_dataframe")
numeric = pd.read_csv("./data/initial_numeric_inputs")
target = pd.read_csv("./data/house_price_target_natlog")

### Construct Model

In [ ]:
all_predictors = pd.concat([numeric, categorical_ohe, categorical_ordinal], axis=1)

In [ ]:
#defining constructor function so it can be used for each iteration

def construct_model(exog_df, endog=target):
    ''' This function takes a dataframe of feature variables, performs robust scaling, 
    and returns summary statistics. '''
   
    rscale = RobustScaler()
    rs = rscale.fit_transform(exog_df)
    rs_df = pd.DataFrame(rs, columns=exog_df.columns, index=exog_df.index)
    
    exog = sm.add_constant(rs_df)
    
    model = sm.OLS(endog, exog).fit().summary()
    return model

# 1st Iteration

In [ ]:
#run 1st model

#concatenate numeric and target to run a correlation
run_corr = pd.concat([target, numeric], axis=1)
run_corr.corr()['ln_price'].sort_values(ascending=False)

Since ln_sqft_living is our highest correlated variable to ln_price, we will use it to run our first regression

In [ ]:
# run first model
construct_model(pd.DataFrame(all_predictors['ln_sqft_living']))

### Evaluate Model Performance

The model accounts for roughly 45% of the target variable variance. ln_sqft_living is statistically signficant with a pvalue less than our alpha at .05. A one unit in iqr of log_sqft_living will increase our log_price by .4845.

### Assess Potential Model Improvements

(1) Increase our rsquared with additional variables

(2) Trim outliers as needed to imrove normality of inputs (JB Score)

# 2nd Iteration

In [ ]:
# create a function that concatenates our next feature variable to our exog dataframe
current_model = pd.DataFrame(all_predictors['ln_sqft_living'])

def iterate(feature):
    feature_df = pd.DataFrame(feature)
    model = pd.concat([current_model, feature_df], axis=1)
    return model

In [ ]:
# run 2nd model adding ln_sqft_living15
current_model = iterate(all_predictors['ln_sqft_living15'])
construct_model(current_model)

### Assess Potential Model Improvements

R2 value was increased, however, some multicollinearity was observed and there are still significant deviations from normality in our inputs overall.

# 3rd Iteration

In [ ]:
# run 3rd model
current_model = iterate(all_predictors['ln_sqft_above'])
construct_model(current_model)

### Assess Potential Model Improvements

R2 indicates our model's correlation has improved marginally, however, normality has also been significantly improved. Feature will be retained.

# 4th Iteration

In [ ]:
# run 4th model
# add bathrooms
current_model = iterate(all_predictors['bathrooms'])
construct_model(current_model)

### Assess Potential Model Improvements¶

R2 value indicates the model has increased correlation strength at the cost of slightly increased collinearity and decreased normality in our inputs. Feature retained.

# 5th Iteration

In [ ]:
# run 4th model
# add bedrooms
current_model = iterate(all_predictors['bedrooms'])
construct_model(current_model)

### Assess Potential Model Improvements¶

R2 and JB have both been significantly improved by the new feature, albeit at a slight cost to collinearity. Feature retained.

# 6th Iteration

In [ ]:
# run 6th model
# add floors
current_model = iterate(all_predictors['floors'])
construct_model(current_model)

### Assess Potential Model Improvements¶

New feature increased R2 value, although it caused increased skew/JB significance and collinearity. Feature retained due to additional increase in kurtosis.

# 7th Iteration

In [ ]:
# run 7th model
# add sqft_lot
current_model = iterate(all_predictors['sqft_lot'])
construct_model(current_model)

In [ ]:
# no change in rqsquared, dropping from exog dataframe
current_model.drop(labels='sqft_lot', axis=1, inplace=True)

### Assess Potential Model Improvements¶

feature dropped from successive models due to an unacceptable increase in collinearity for a negligible increase in correlation strength or other parameters.

# 8th Iteration

In [ ]:
# run 8th model
# add sqft_lot15
current_model = iterate(all_predictors['sqft_lot15'])
construct_model(current_model)

In [ ]:
# no change in rqsquared, dropping from exog dataframe
current_model.drop(labels='sqft_lot15', axis=1, inplace=True)

### Assess Potential Model Improvements¶

feature dropped from successive models due to an unacceptable increase in collinearity for a negligible increase in correlation strength or other parameters.

# 9th Iteration

In [ ]:
# run 9th model
# add renovated
current_model = iterate(all_predictors['renovated'])
construct_model(current_model)

In [ ]:
# no change in rqsquared, dropping from exog dataframe
current_model.drop(labels='renovated', axis=1, inplace=True)

### Assess Potential Model Improvements

feature dropped from successive models due to an unacceptable increase in collinearity for a negligible increase in correlation strength or other parameters.

# 10th Iteration

In [ ]:
# run 10th model
# add conditional ordinal encoded
current_model = iterate(categorical_ordinal['cond_ord'])
construct_model(current_model)

### Assess Potential Model Improvements

Significant increase in the model's correlation strength and slight increase in skew. Feature retained due to increased kurtosis and correlation strength.

# 11th Iteration

In [ ]:
# run 11th model
# add grade ordinal encoded
current_model = iterate(categorical_ordinal['grade_ord'])
construct_model(current_model)

### Assess Potential Model Improvements

signficant improvement in all metrics except for collinearity. Model is still within limits of collinearity assumption (Cond. No. <10), so feature is retained.

# 12th Iteration

In [ ]:
# run 12th model
# removes condition of house variable to assess impact on collinearity
current_model.drop(labels='cond_ord', axis=1, inplace=True)
construct_model(current_model)

### Assess Potential Model Improvements

Model suffered penalties to correlation strength without improving collinearity. Feature will be retained in possible final model.

# 13th Iteration

In [ ]:
# run 13th model
# add grade ordinal encoded
# assessing whether one-hot-encoding offers performance benefits compared to ordinal encoding for 
# categorical variables/features.
current_model.drop(labels='grade_ord', axis=1, inplace=True)
ohe_df = pd.concat([current_model, categorical_ohe], axis=1)
construct_model(ohe_df)

### Assess Potential Model Improvements

Replacing ordinal encoding with one-hot-encoding results in mild improvement to correlation strength, but also results in unacceptable collinearity increase.

# 14th Iteration

In [ ]:
# run 13th model
# add grade ordinal encoded
ohe_df.drop(labels=['x0_Fair', 'x0_Good', 'x0_Poor', 'x0_Very Good'], axis=1, inplace=True)
construct_model(ohe_df)

### Assess Potential Model Improvements

Similar trade-offs observed with 13th and 14th models. Due to collinearity increase, ordinal encoding is preferred for the final model despite correlation strength improvement.

# Preparing the Final Model

In [ ]:
#12th iteration was the best fit we could produce with our available features without violating assumptions
#here we recreate that dataset for use in the final model
semi_final_df = pd.concat([target, numeric, categorical_ordinal], axis=1)

In [ ]:
semi_final_df.drop(labels=['sqft_lot', 'age', 'renovated', 'sqft_lot15'], axis=1, inplace=True)

In [ ]:
#QQ plot generated for each feature to identify outliers that may need to be dropped before 
#the final model is created
for e, f in enumerate(semi_final_df.columns):
    sm.qqplot(semi_final_df[f], line='r');
    plt.title(f)

QQ plot indicates that bedrooms are significantly affected by outliers, and bathrooms may be significantly skewed/non-normally distributed. Final iteration of our model will experiment with removal of bathrooms as a feature and trimming outliers from the dataset that are affecting bedroom regression modeling. 

### Additional feature cleaning

In [ ]:
#identify houses with the most bedrooms (top 10)
bed_10 = semi_final_df.sort_values(by="bedrooms", ascending=False)[0:10]
bed_10

In [ ]:
#since the house with 33 bedrooms has a smaller number of bathrooms than houses with 1/3 the number of 
#bedrooms, it is likely that this data is actually a typo and is significantly skewing the data.
#Therefore, we will drop this house from the overall dataset before constructing the final model.
#index position is 15856, however, we will reevaluate to ensure we dropped the correct data point

semi_final_df.drop(index=15856, inplace=True)
#reevaluate
bed_10 = semi_final_df.sort_values(by="bedrooms", ascending=False)[0:10]
bed_10

### Final Model Development

In [ ]:
semi_final_df.head()

In [ ]:
#experimenting with dropping the bathrooms feature due to non-normality and heavy presence of outliers
new_target = semi_final_df["ln_price"]
final_inputs = semi_final_df.drop(columns=["bathrooms","ln_price"])
construct_model(final_inputs, new_target)

In [ ]:
#final comparison model with bathroom data retained
new_target = semi_final_df["ln_price"]
final_inputs = semi_final_df.drop(columns=["ln_price"])
construct_model(final_inputs, new_target)

# Results

In [ ]:
# The Final Final Model
final_target_df = semi_final_df["ln_price"]
final_inputs_df = semi_final_df.drop(columns=["bathrooms","ln_price"])

#construct final model and prep data for export
#scale the data
rscale = RobustScaler()
rs = rscale.fit_transform(final_inputs_df)
rs_df = pd.DataFrame(rs, columns=final_inputs_df.columns, index=final_target_df.index)

#final construction    
exog = sm.add_constant(rs_df)
endog = final_target_df
final_model = sm.OLS(endog, exog).fit()

#final dataframe prepping for export (1) rename scaled data (2) join all 3 dataframes
#1
rs_df.rename(columns={name:f'scl_{name}' for name in rs_df.columns}, inplace=True)
#2
final_df = pd.concat([final_target_df, final_inputs_df, rs_df], axis=1)

#generating summary statistics
fmss = final_model.summary()
fmss

In [ ]:
prediction = final_model.predict(exog)
prediction # scatter endog and pred 
#polyfit to draw line (predicted, actual)

In [ ]:
# extract coefficients table
accursed_table = pd.DataFrame(fmss.tables[1].data[0:])
accursed_table

In [ ]:
#replace columns with the appropriate headers
new_header = accursed_table.iloc[0]
accursed_table = accursed_table[1:]
accursed_table.columns = new_header
accursed_table

In [ ]:
#sort table by coefficients
blursed_table = accursed_table.sort_values(by="coef", ascending=False)
blursed_table

In [ ]:
#Top two features increasing home sale price in 2014-2015 were ln_sqft_living, and grade_ord 
#from the assessors office for King County, Washington

### Visualizations and Exports

In [ ]:
final_df.columns

In [ ]:
#visualize model for living space correlation

#variable declaration: see model summary for coefficient and intercept values
x = final_df["ln_sqft_living"]
y = final_df["ln_price"]
m = 0.3188
b0 = 12.8938
y_ticks_liv = [11, 12, 13, 14, 15, 16]
x_ticks_liv = [6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5]

fig, ax = plt.subplots(figsize=(10,10))
fig = plt.scatter(x=x,y=y, color="#34b456")

ax.set_xlabel('Sqft Living Space (Log unit)', fontsize=24, labelpad=20) 
ax.set_ylabel('House Unit Sale Price (Log USD)', fontsize=24, labelpad=25) 
ax.set_title('Living Space vs. House Sale Price', fontsize=30, pad=15); 

ax.set_ylim(11, 16, 1)
ax.set_yticks(y_ticks_liv)
ax.set_yticklabels(y_ticks_liv, fontsize=17)
ax.set_xlim(6, 9.5, 1)
ax.set_xticks(x_ticks_liv)
ax.set_xticklabels(x_ticks_liv, fontsize=17)

plt.savefig('./Results/Sqft_Living_v_Sale_Price.jpg')

In [ ]:
#visualize the raw correlation of living space v house sales (non-transformed)
#variable declaration: see model summary for coefficient and intercept values
x = np.exp(final_df["ln_sqft_living"])
y = np.exp(final_df["ln_price"])
m = 0.3188
b0 = 12.8938

fig, ax = plt.subplots(figsize=(10,10))
fig = plt.scatter(x=x,y=y, color="green")

ax.set_xlabel('Sqft Living Space', fontsize=24, labelpad=20) 
ax.set_ylabel('House Sale Price (USD)', fontsize=24, labelpad=25) 
ax.set_title('Living Space vs. House Sale Price (Raw)', fontsize=30, pad=15); 

x_val=[f"{x*2},000" for x in range(-1,8)]
y_val=[f"${y},000,000" for y in range(0,9)]
ax.set_xticks=[x*2000 for x in range(-1,8)]
ax.set_yticks=[y*1000000 for y in range(0,9)]
ax.set_xticklabels(x_val, fontsize=17)
ax.set_yticklabels(y_val, fontsize=17)

plt.savefig('./Results/Raw_Sqft_Living_v_Sale_Price.jpg')

In [ ]:
#visualize model overall performance

# Change font family
plt.rcParams.update({'font.family':'Gill Sans'})

fig, ax = plt.subplots(figsize=(10,10))
fig = plt.scatter(x=prediction,y=endog, color="red")
y_ticks = [11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16]
x_ticks = [11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15]

ax.set_xlabel('Predicted Unit House Sales (Log USD)', fontsize=24, labelpad=20) #color='white
ax.set_ylabel('Actual Unit House Sales (Log USD)', fontsize=24, labelpad=25) #color='white'
ax.set_title('Actual vs Predicted House Sales', fontsize=30, pad=15) #color='white'

ax.set_ylim(11, 16, .5)
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticks, fontsize=17)
ax.set_xlim(11.5, 15, .5)
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticks, fontsize=17, rotation=45)

#ax.set_xticklabels(prediction, fontsize=15, rotation=45); #color='white'

m, b0 = np.polyfit(prediction, endog, deg=1)
plt.plot(prediction, m*prediction+b0);
plt.savefig('./Results/Actual_v_Predicted.jpg')

In [ ]:
#visualize model for living space correlation

fig, ax = plt.subplots(figsize=(10,10))
fig = plt.scatter(x=np.exp(prediction),y=np.exp(endog), color="red")

ax.set_xlabel('Predicted House Sales (Millions USD)', fontsize=24, labelpad=20) #color='white
ax.set_ylabel('Actual House Sales (Millions USD)', fontsize=24, labelpad=25) #color='white'
ax.set_title('Actual vs Predicted House Sales (Raw)', fontsize=30, pad=15) #color='white'

x_val=[f"${x/2}" for x in range(-1,7)]
y_val=[f"${y}" for y in range(0,9)]
ax.set_xticks=[(x/2)*1000000 for x in range(-1,7)]
ax.set_yticks=[y*1000000 for y in range(0,9)]
ax.set_xticklabels(x_val, fontsize=17, rotation=45)
ax.set_yticklabels(y_val, fontsize=17)

m, b0 = np.polyfit(np.exp(prediction), np.exp(endog), deg=1)
plt.plot(np.exp(prediction), m*(np.exp(prediction))+b0);
plt.savefig("./Results/Raw_Actual_v_Predicted.jpg")

In [ ]:
coef_df = pd.DataFrame({'coef': {'Sqft Above': -0.1164, 'Bedrooms': -0.0303, 
                        'Sqft Living': 0.3188, 'Grade': 0.2023, 'Sqft Living 15': 0.1017, 
                        'Condition': 0.1010, 'Floors': 0.0654}})
coef_sorted_df = coef_df.sort_values(by='coef', ascending=False).reset_index()
coef_sorted_df

In [ ]:
#vizualize model for coefficients

# Create variables for chart
coef_x = coef_sorted_df['index']
coef_y = coef_sorted_df['coef']
y_ticks = [-.2, -.1, 0, .1, .2, .3, .4]
c = ['#34b456', '#34b456', '#747474', '#747474', '#747474', '#747474', '#747474']
# Change font family
plt.rcParams.update({'font.family':'Gill Sans'})

# Create bar chart
fig, ax = plt.subplots(figsize = (20,10))

bars = ax.bar(coef_x, coef_y, color=c)
ax.set_xlabel('Feature Variabes', fontsize=27, labelpad=20) #color='white'
ax.set_ylabel('Coefficient', fontsize=27, labelpad=25) #color='white'
ax.set_title('Feature Variables Effect on House Sale Price', fontsize=30, pad=15) #color='white'
ax.set_ylim(-.2, .4, 1)
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticks, fontsize=20) #color='white'
ax.set_xticks(coef_x)
ax.set_xticklabels(coef_x, fontsize=20, rotation=45); #color='white'
plt.savefig("./Results/Coeff_Bar_Chart.jpg")

In [ ]:
pd.DataFrame(final_df.groupby(by='grade_ord', axis=0).mean()['ln_price'])

In [ ]:
grade_ord_viz_df = pd.DataFrame({'Grade': {'Poor': 12.1112, 'Low': 12.1602, 
                        'Fair': 12.3233, 'Low Average': 12.5438, 'Average': 12.8365, 
                        'Good': 13.1362, 'Better': 13.4874, 'Very Good': 13.8030, 'Excellent': 14.1319, 'Luxury': 14.5147,
                        'Mansion': 15.0282}}).reset_index()
grade_ord_sort_viz_df = grade_ord_viz_df.sort_values(by='Grade', ascending=True)

In [ ]:
grade_ord_sort_viz_df

In [ ]:
#visualize model for assessor grade correlation
#vizualize model for coefficients

# Create variables for chart
grade_ord_x = grade_ord_sort_viz_df['index']
ln_price_y = grade_ord_sort_viz_df['Grade']
y_ticks = [12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5]
c = ['#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456']
# Change font family
plt.rcParams.update({'font.family':'Gill Sans'})

# Create bar chart
fig, ax = plt.subplots(figsize = (15,10))

bars = ax.bar(grade_ord_x, ln_price_y, color=c)
ax.set_xlabel('Grade Values', fontsize=27, labelpad=20, color='white') #color='white'
ax.set_ylabel('Average Unit Sale Price (Log USD)', fontsize=27, labelpad=25, color='white') #color='white'
ax.set_title('House Grade vs Average Sale Price', fontsize=30, pad=15, color='white') #color='white'
ax.set_ylim(12, 13, .5)
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticks, fontsize=20, color='white') #color='white'
ax.set_xticks(grade_ord_x)
ax.set_xticklabels(grade_ord_x, fontsize=20, rotation=45, color='white'); #color='white'

plt.savefig('./Results/Grade_vs_Price.jpg')

In [ ]:
#visualize model for assessor grade correlation (raw price)
#vizualize model for coefficients

# Create variables for chart
grade_ord_x = grade_ord_sort_viz_df['index']
ln_price_y = np.exp(grade_ord_sort_viz_df['Grade'])
y_ticks = [y/2 for y in range(0,8)]
c = ['#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456', '#34b456']

# Create bar chart
fig, ax = plt.subplots(figsize = (15,10))

bars = ax.bar(grade_ord_x, ln_price_y, color=c)
ax.set_xlabel('Grade Values', fontsize=27, labelpad=20, color='white') #color='white'
ax.set_ylabel('Average Sale Price (Millions USD)', fontsize=27, labelpad=25, color='white') #color='white'
ax.set_title('House Grade vs Average Sale Price (Raw)', fontsize=30, pad=15, color='white') #color='white'

ax.set_yticks([y*1000000 for y in y_ticks])
ax.set_yticklabels(y_ticks, fontsize=20, color='white') #color='white'
ax.set_xticks(grade_ord_x)
ax.set_xticklabels(grade_ord_x, fontsize=20, rotation=45, color='white'); #color='white'

plt.savefig('./Results/Raw_Grade_vs_Price.jpg')

In [ ]:
final_df.info()

In [ ]:
final_df.groupby(by='grade_ord', axis=0).mean()['ln_price']